In [1]:
%load_ext autoreload
%autoreload 2
from models.factory.ModelFactory import ModelFactory
from evaluation.eval_vqa import VQA_Evaluator
from dataset.VQAv2Dataset import VQAv2Dataset
from torch.utils.data import DataLoader
import json
import os
import torch
import tqdm

In [2]:
model_factory = ModelFactory()
config = model_factory.create_config('frcnn_concat')
test_dataset = VQAv2Dataset(
        split="test",
        txt_enc=config['txt_enc'],
        bottom_up_features_dir=config['bottom_up_features_dir'],
        skipthoughts_dir=config['skipthoughts_dir'],
        processed_dir=config['processed_dir'],
        ROOT_DIR=config['ROOT_DIR'],
        vqa_dir=config['vqa_dir'])

test_loader = DataLoader(
        test_dataset,
        shuffle=True,
        batch_size=1024,
        num_workers=config['num_workers'],
        collate_fn=test_dataset.collate_fn)

# Construct word vocabulary
word_vocabulary = [word for _,word in test_dataset.word_to_wid.items()]


model = model_factory.create_model('frcnn_concat', config, word_vocabulary)
model = model.cuda()


/auto/homes/bat34/VQA_PartII/models/factory/ModelFactory.py:34: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)


In [3]:
OLD_TRAINED_FILES = '/local/scratch/bat34/baseline_trained_models/best_models/{}/best_model.pth'
#OLD_TRAINED_FILES = '/home/bat34/VQA_PartII/models/baseline/trained_models/best_models/{}/best_model.pth'

In [4]:
MODEL_NAME = 'agg_concat_min_seed_2048_agg_type_min_q_self_attention_True_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0003_fusion_type_concat_mlp'

In [5]:
MODEL_SAVE_NAME = OLD_TRAINED_FILES.format(MODEL_NAME)
best_ckpt = torch.load(MODEL_SAVE_NAME)
model.load_state_dict(best_ckpt['model'])

<All keys matched successfully>

In [6]:
def evaluate(config, model, val_loader,
                 evaluator, aid_to_ans,
                 RESULTS_FILE_PATH, device):
    model.eval()
    print('Running model on validation dataset..')
    with torch.no_grad():
        results = []
        total_batch_loss = 0
        batch_iter = 0
        for data in tqdm.tqdm(val_loader):
            item = {
                    'question_ids': data['question_ids'].cuda(),
                    'object_features_list': data['object_features_list'].cuda(),
                    'bounding_boxes': data['bounding_boxes'].cuda(),
                    'question_lengths': data['question_lengths'].cuda()
            }

            if 'murel' in config['name'] and config['use_graph_module']:
                item['graph_batch'] = data['graph'].to(device)

            inputs = item
            qids = data['question_unique_id']
            outputs = model(inputs)

            values, ans_indices = torch.max(outputs, dim=1)
            ans_indices = list(ans_indices)
            ans_indices = [tsr.item() for tsr in ans_indices]
            for qid, ans_idx in zip(qids, ans_indices):
                results.append({
                    'question_id': int(qid),
                    'answer': aid_to_ans[ans_idx]
                })
            batch_iter += 1

    print('Finished evaluating the model on the val dataset.')
    print('Saving results to %s' % RESULTS_FILE_PATH)
    with open(RESULTS_FILE_PATH, 'w') as f:
        json.dump(results, f)
    print('Done saving to %s' % RESULTS_FILE_PATH)
#     print('Calling VQA evaluation subroutine')
#     # We let the evaluator do all the tensorboard logging for accuracy
#     accuracy = evaluator.evaluate(RESULTS_FILE_PATH, 1)
#     print("Validation Results  Overall  accuracy: {:.2f}".format( accuracy))

In [7]:
evaluator = None
SAVE_FILE = '/home/bat34/VQA_PartII/models/'
SAVE_NAME = '/local/scratch/bat34/test_result_files/{}.json'.format(MODEL_NAME)
SAVE_NAME = SAVE_NAME if len(SAVE_NAME) <= 100 else SAVE_NAME[]
evaluate(config, model, test_loader, evaluator, test_dataset.aid_to_ans, 
            SAVE_NAME, 'cuda')

  0%|          | 0/438 [00:00<?, ?it/s]

Running model on validation dataset..


/home/bat34/venv/lib/python3.5/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/home/bat34/venv/lib/python3.5/site-packages/torch/nn/functional.py:1340: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
100%|██████████| 438/438 [24:09<00:00,  3.31s/it] 


Finished evaluating the model on the val dataset.
Saving results to /local/scratch/bat34/test_result_files/agg_concat_min_seed_2048_agg_type_min_q_self_attention_True_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0003_fusion_type_concat_mlp.json
Done saving to /local/scratch/bat34/test_result_files/agg_concat_min_seed_2048_agg_type_min_q_self_attention_True_txt_enc_BayesianUniSkip_batch_size_256_lr_0.0003_fusion_type_concat_mlp.json


NameError: name 'accuracy' is not defined